# Space Invaders
### DEPENDENCIES:

1. gym & gym[atari] v.0.19.0
2. pyglet version v.1.5.11
3. stable_baselines3 1.3.0
4. SWIG v.3.0.12 (http://www.swig.org/download.html)
5. Keras-RL2 1.0.5
6. tensorflow v.1.14.0
7. ale-py 0.7.3

#### Microsoft Visual C++ 14.0 or greater is required.

In [ ]:
!pip install gym
!pip install gym[atari]
!pip install pyglet==1.5.11
!pip install stable_baselines3
!pip install keras-rl2
!pip install tensorflow==1.14.0
!pip install ale-py

# 1. Test Random Environment with OpenAI Gym

In [ ]:
import gym 
import random

In [ ]:
import gym 
import random
# defining the environment
env = gym.make('SpaceInvaders-v0')
# grabbing the height, width and channels from the environment frame to construct the neural network
height, width, channels = env.observation_space.shape
# grabbing all the possible actions the agent can do
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        # action = random.choice([0,1,2,3,4,5])
        action = env.action_space.sample()
        # extract info from each episode
        n_state, reward, done, info = env.step(action)
        # append episode score to total score
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
# Sequencial deep learning model
from tensorflow.keras.models import Sequential
# Convolution to scan the performance
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
# Optimizer Algorithm
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(height, width, channels, actions):
    # Started Sequential API with convolution 32 filters, 8x8matrixes with 4x4stride, relu function, 3 frames  
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    # Adding more layers with different convolution matrixes
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    # Flatten all convolution layers into one
    model.add(Flatten())
    # Dense layers (aka fully connected layers), every unit in a layer is connected to every single unit the next one
    # 512 Units, relu activation
    model.add(Dense(512, activation='relu'))
    # Compress dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model

In [ ]:
# Building model
model = build_model(height, width, channels, actions)

In [ ]:
# Model Architecture
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
# Reinforcement Learning Agent
from rl.agents import DQNAgent
# Knowledge buffer, retain information from previous games
from rl.memory import SequentialMemory
# When close to optimal strategy it starts closing / Best Reward Outcome
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [ ]:
def build_agent(model, actions):
    # 
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    # Buffer limit, store the past 3 windows for 1000 episodes 
    memory = SequentialMemory(limit=1000, window_length=3)
    
    # dueling network (competing network or reinforcement learning model): 
    # splits Value/Advantage, helps the model learn when to take action or when not to bother
    # can learn which states are (or are not) valuable, without having to learn the effect of each action for each state. 
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=10000)
    return dqn

In [ ]:
# Compile Agent with Adam Optimizer with a 0.0001 learning rate
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-4))

In [ ]:
# fit model with 1000 episodes 
# Verbose 2: Visualize training output every couple episodes ( includes 1 -> live progress bar)
# State of the art model at 10,000,000 steps
dqn.fit(env, nb_steps=1000, visualize=False, verbose=2)

In [ ]:
scores = dqn.test(env, nb_episodes=10, visualize=True, verbose=2)
print(np.mean(scores.history['episode_reward']))

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

In [ ]:
del model, dqn

In [ ]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')